In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src/')

In [2]:
from package.logger import Timed, rlog, setup
from package import storage

setup("INFO")

In [3]:
import os

import folium
import geopandas as gpd
import pandas as pd

from package import strtime
from package.osm import osm
from package.mcr.data import NetworkType, OSMData
from package.mcr5.mcr5 import MCR5
from package import storage

from package.geometa import GeoMeta
from package.mcr.data import NetworkType

In [4]:
geo_meta_path = "../data/geometa.pkl"
city_id = "Koeln"
stops = "../data/cleaned/stops.csv"
structs = "../data/structs.pkl"
# bicycle_location_path = "../data/bicycle_locations/2022-11-01_09_01_00.csv"

In [5]:
geo_meta = GeoMeta.load(geo_meta_path)
# geo_data = OSMData(
# 	geo_meta,
# 	city_id,
# 	additional_network_types=[NetworkType.DRIVING, NetworkType.CYCLING],
# )
# nodes = geo_data.osm_nodes
# edges = geo_data.osm_edges
# driving_nodes, _, _ = geo_data.additional_networks[NetworkType.DRIVING]

In [6]:
lm_data = storage.read_any_dict("../data/location_mappings.pkl")
location_mappings = lm_data["location_mappings"]
car_location_mappings = lm_data["car_location_mappings"]

In [7]:
len(location_mappings)

1782

In [8]:
from command.step_config import (
    get_bicycle_public_transport_config,
    get_car_only_config,
    get_bicycle_only_config,
    get_public_transport_only_config,
    get_walking_only_config,
)

In [9]:
configs = {}


def get_bicyle_public_transport_config_ready(bicycle_location_path, start_time):
    initial_steps, repeating_steps = get_bicycle_public_transport_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
        bicycle_price_function="next_bike_tariff",
        bicycle_location_path=bicycle_location_path,
        structs_path=structs,
        stops_path=stops,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
        "start_time": start_time,
    }


def get_car_only_config_ready():
    initial_steps, repeating_steps = get_car_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": car_location_mappings,
        "max_transfers": 1,
    }


def get_bicycle_only_config_ready(bicycle_location_path):
    initial_steps, repeating_steps = get_bicycle_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
        bicycle_price_function="next_bike_tariff",
        bicycle_location_path=bicycle_location_path,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
    }


def get_public_transport_only_config_ready(start_time):
    initial_steps, repeating_steps = get_public_transport_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
        structs_path=structs,
        stops_path=stops,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
        "start_time": start_time,
    }


def get_walking_only_config_ready():
    initial_steps, repeating_steps = get_walking_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 0,
    }

In [10]:
import itertools
import os

times = [
    "08:00:00",
    "12:00:00",
    "18:00:00",
]
bicycle_base_path = "../data/bicycle_locations/"
bicycle_location_paths = [
    os.path.join(bicycle_base_path, path)
    for path in os.listdir("../data/bicycle_locations/")
]

bicycle_public_transport_config_args = list(itertools.product(
	times,
	bicycle_location_paths,
))

In [11]:
from functools import partial

for i, (time, bicycle_location_path) in enumerate(bicycle_public_transport_config_args):
    print(i, time, bicycle_location_path)
    configs[f"bicycle_public_transport_{i}"] = partial(
        get_bicyle_public_transport_config_ready, bicycle_location_path, time
    )

print()
for i, time in enumerate(times):
    print(i, time)
    configs[f"public_transport_{i}"] = partial(
        get_public_transport_only_config_ready, time
    )

print()
for i, bicycle_location_path in enumerate(bicycle_location_paths):
    print(i, bicycle_location_path)
    configs[f"bicycle_{i}"] = partial(
        get_bicycle_only_config_ready, bicycle_location_path
    )

# configs["car"] = get_car_only_config_ready
# configs["walking"] = get_walking_only_config_ready

0 08:00:00 ../data/bicycle_locations/2022-11-02_16_01_00.csv
1 08:00:00 ../data/bicycle_locations/2022-11-01_09_01_00.csv
2 08:00:00 ../data/bicycle_locations/2022-07-18_06_01_00.csv
3 08:00:00 ../data/bicycle_locations/2022-04-19_17_01_00.csv
4 12:00:00 ../data/bicycle_locations/2022-11-02_16_01_00.csv
5 12:00:00 ../data/bicycle_locations/2022-11-01_09_01_00.csv
6 12:00:00 ../data/bicycle_locations/2022-07-18_06_01_00.csv
7 12:00:00 ../data/bicycle_locations/2022-04-19_17_01_00.csv
8 18:00:00 ../data/bicycle_locations/2022-11-02_16_01_00.csv
9 18:00:00 ../data/bicycle_locations/2022-11-01_09_01_00.csv
10 18:00:00 ../data/bicycle_locations/2022-07-18_06_01_00.csv
11 18:00:00 ../data/bicycle_locations/2022-04-19_17_01_00.csv

0 08:00:00
1 12:00:00
2 18:00:00

0 ../data/bicycle_locations/2022-11-02_16_01_00.csv
1 ../data/bicycle_locations/2022-11-01_09_01_00.csv
2 ../data/bicycle_locations/2022-07-18_06_01_00.csv
3 ../data/bicycle_locations/2022-04-19_17_01_00.csv


In [12]:
mcr5_output_path = "../data/mcr5/Koeln-tariffs"

In [13]:
# configs = {
# 	"bicycle_0": configs["bicycle_0"],
# 	"bicycle_1": configs["bicycle_1"],
# 	"bicycle_2": configs["bicycle_2"],
# 	"bicycle_3": configs["bicycle_3"],
# }

In [14]:
if os.path.exists(mcr5_output_path):
	raise Exception("Output path already exists")

In [15]:
from datetime import datetime
import pickle

In [16]:
configs

{'bicycle_public_transport_0': functools.partial(<function get_bicyle_public_transport_config_ready at 0x7f33b42a4430>, '../data/bicycle_locations/2022-11-02_16_01_00.csv', '08:00:00'),
 'bicycle_public_transport_1': functools.partial(<function get_bicyle_public_transport_config_ready at 0x7f33b42a4430>, '../data/bicycle_locations/2022-11-01_09_01_00.csv', '08:00:00'),
 'bicycle_public_transport_2': functools.partial(<function get_bicyle_public_transport_config_ready at 0x7f33b42a4430>, '../data/bicycle_locations/2022-07-18_06_01_00.csv', '08:00:00'),
 'bicycle_public_transport_3': functools.partial(<function get_bicyle_public_transport_config_ready at 0x7f33b42a4430>, '../data/bicycle_locations/2022-04-19_17_01_00.csv', '08:00:00'),
 'bicycle_public_transport_4': functools.partial(<function get_bicyle_public_transport_config_ready at 0x7f33b42a4430>, '../data/bicycle_locations/2022-11-02_16_01_00.csv', '12:00:00'),
 'bicycle_public_transport_5': functools.partial(<function get_bicyle_

In [17]:
runtimes = {}
for key, config in configs.items():
    start = datetime.now()
    rlog.info(f"Running MCR5 for {key}")

    with Timed.info("Loading config"):
        config = config()
        mcr5 = MCR5(**config["init_kwargs"], max_processes=8)

    output_path = os.path.join(mcr5_output_path, key)

    location_mappings = config["location_mappings"]

    with Timed.info("Running MCR5"):
        start_time = config.get("start_time", "08:00:00")
        errors = mcr5.run(
            location_mappings,
            start_time=start_time,
            output_dir=output_path,
            max_transfers=config["max_transfers"],
        )
    total_time = datetime.now() - start
    runtimes[key] = total_time

with open(os.path.join(mcr5_output_path, "runtimes.pkl"), "wb") as f:
    pickle.dump(runtimes, f)

[12:35:49] INFO     Running MCR5 for bicycle_public_transport_0                                     ]8;id=224233;file:///tmp/ipykernel_1709698/1400244749.py\1400244749.py]8;;\:]8;id=569520;file:///tmp/ipykernel_1709698/1400244749.py#4\4]8;;\

           INFO     Loading config                                                                  ]8;id=490988;file:///tmp/ipykernel_1709698/1400244749.py\1400244749.py]8;;\:]8;id=87893;file:///tmp/ipykernel_1709698/1400244749.py#6\6]8;;\

           INFO     Using existing OSM data                                                         ]8;id=276982;file:///tmp/ipykernel_1709698/1400244749.py\1400244749.py]8;;\:]8;id=30980;file:///tmp/ipykernel_1709698/1400244749.py#6\6]8;;\

           INFO     Loading OSM network from cache (walking)                                        ]8;id=261973;file:///tmp/ipykernel_1709698/1400244749.py\1400244749.py]8;;\:]8;id=96373;file:///tmp/ipykernel_1709698/1400244749.py#6\6]8;;\

[12:36:53] INFO     Using existing OSM data                                                         ]8;id=524428;file:///tmp/ipykernel_1709698/1400244749.py\1400244749.py]8;;\:]8;id=433517;file:///tmp/ipykernel_1709698/1400244749.py#6\6]8;;\

...

[19:06:01] INFO     Loading config done (4:11 minutes)                                              ]8;id=397557;file:///tmp/ipykernel_1709698/1400244749.py\1400244749.py]8;;\:]8;id=84221;file:///tmp/ipykernel_1709698/1400244749.py#6\6]8;;\

           INFO     Running MCR5                                                                   ]8;id=135063;file:///tmp/ipykernel_1709698/1400244749.py\1400244749.py]8;;\:]8;id=446696;file:///tmp/ipykernel_1709698/1400244749.py#14\14]8;;\

Starting:   0%|          | 0/1782 [00:00<?, ?it/s]

[19:22:32] INFO     All processes finished.                                                        ]8;id=644477;file:///tmp/ipykernel_1709698/1400244749.py\1400244749.py]8;;\:]8;id=325400;file:///tmp/ipykernel_1709698/1400244749.py#14\14]8;;\

[19:22:33] INFO     Running MCR5 done (16:31 minutes)                                              ]8;id=765664;file:///tmp/ipykernel_1709698/1400244749.py\1400244749.py]8;;\:]8;id=195837;file:///tmp/ipykernel_1709698/1400244749.py#14\14]8;;\

In [18]:
import pickle

# errors = storage.read_any_dict("../data/mcr5/Koeln/bicycle_public_transport/errors.pkl")
# errors